# The Blessing and Curse of Non-determinism

One of the most fundamental challenges when building products with Large Language Models (LLMs) is their non-deterministic nature. Unlike traditional software systems where the same input reliably produces the same output, LLMs can generate different responses each time they're queried - even with identical prompts. This characteristic is both a strength and a significant engineering challenge.

## Understanding the Challenge

### What is Non-determinism in LLMs?

When you ask ChatGPT or any other LLM the same question multiple times, you'll likely get different responses. This isn't a bug - it's a fundamental feature of how these models work. The "temperature" parameter, which controls the randomness of outputs, allows models to be creative and generate diverse responses. However, this same feature makes it incredibly difficult to build reliable, testable systems.

### Real-world Impact

Consider a financial services company using LLMs to generate investment advice summaries. The non-deterministic nature of these models means that:
- The same market data could yield different analysis conclusions
- Testing becomes exponentially more complex
- Regulatory compliance becomes challenging to guarantee
- User trust may be affected by inconsistent responses

## Technical Deep-dive: Sources of Non-determinism

### Temperature and Sampling

The primary source of non-determinism in LLMs comes from their sampling strategies. During text generation, the model:
1. Calculates probability distributions for each next token
2. Samples from these distributions based on temperature settings
3. Uses techniques like nucleus sampling to balance creativity and coherence

### The Temperature Spectrum

- Temperature = 0: Most deterministic, but potentially repetitive
- Temperature = 1: Balanced creativity and coherence
- Temperature > 1: Increased randomness, potentially incoherent

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

from openai import OpenAI
import pandas as pd
from typing import List

def generate_responses(
    prompt: str,
    temperatures: List[float],
    attempts: int = 3
) -> pd.DataFrame:
    """
    Generate multiple responses at different temperature settings
    to demonstrate non-deterministic behavior.
    """
    client = OpenAI()
    results = []
    
    for temp in temperatures:
        for attempt in range(attempts):
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=temp,
                max_tokens=50
            )
            
            results.append({
                'temperature': temp,
                'attempt': attempt + 1,
                'response': response.choices[0].message.content
            })
    
    return pd.DataFrame(results)

def demonstrate_temperature_effects(prompt: str,
                                    attempts: int = 3, 
                                    temperatures: List[float] = [0.0, 1.0, 2.0]):
    """
    Run a demonstration of temperature effects.
    """
    results = generate_responses(prompt, temperatures, attempts)
    
    # Display results grouped by temperature
    for temp in temperatures:
        print(f"\nTemperature = {temp}")
        print("-" * 40)
        temp_responses = results[results['temperature'] == temp]
        for _, row in temp_responses.iterrows():
            print(f"Attempt {row['attempt']}: {row['response']}")
    return results

In [4]:
df_results = demonstrate_temperature_effects("Suggest a name for a coffee shop", temperatures=[0.0, 1.0, 2.0])


Temperature = 0.0
----------------------------------------
Attempt 1: "Café Perk"
Attempt 2: "Café Perk"
Attempt 3: "Café Perk"

Temperature = 1.0
----------------------------------------
Attempt 1: "Bean Bliss Cafe"
Attempt 2: "Caffeine Haven"
Attempt 3: "Café Haven"

Temperature = 2.0
----------------------------------------
Attempt 1: "Caffeine Chronicles"
Attempt 2: Earthbean Café
Attempt 3: "Beans & Birch: Grab a Brew"
